In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request as urllib
from requests import get
import matplotlib.pyplot as plt
from dateutil import parser
%matplotlib inline

In [41]:
from apscheduler.schedulers.blocking import BlockingScheduler
from datetime import datetime, date
import os, glob

In [4]:
sched = BlockingScheduler()# call the BlockingScheduler class

Now we will define the job that needs to be run periodically. In this case the job will consist of going through a list of urls from which it will extract whether an item which is displayed online for sale is available or not. The scheduler should open a csv file and input the availability of the item. 

In [5]:
#get the list of urls
#Given a base url, this item will keep extracting URLs from the search result page.
base_url = 'https://www.ebay.com/b/Camping-Hiking-Sleeping-Gear/181403/bn_7327730?_pgn='
## this function returns a list of links correspoding to an item on sale in ebay
def get_item_links(base_url, uppercut = 800000):
    links = []
    count = 1
    while len(links)<uppercut:
        url = base_url+str(count)
        response = get(url)
        print(url)
        html_soup = BeautifulSoup(response.text, 'html.parser')
        item_page_links = html_soup.find_all('div', class_ = "s-item__wrapper clearfix")
        for link in item_page_links:
            links.append(link.a.attrs['href'])
        count+=1
    return links[:uppercut]
    

#print(response.text[500:1000])

In [6]:
#here we will call the function to get the desired number of links, in order to keep things simple we will just get
#5 links
item_urls = get_item_links(base_url, uppercut=5)

https://www.ebay.com/b/Camping-Hiking-Sleeping-Gear/181403/bn_7327730?_pgn=1


['https://www.ebay.com/itm/Inflatable-Air-Sofa-Bed-Lazy-Sleeping-Camping-Bag-Beach-Hangout-Couch-Windbed/292566375653?epid=715012867&hash=item441e509ce5:m:mSeLLE_HXu8OCpTWY6uyAZA&var=591377547355',
 'https://www.ebay.com/itm/Double-Self-Inflating-Camping-Roll-Mat-Camp-Bed-Inflatable-Sleeping-Mattress/253747098737?hash=item3b14817471:g:brEAAOSwZrdbRxom',
 'https://www.ebay.com/itm/New-Double-Outdoor-Person-Travel-Camping-Hanging-Hammock-Bed-Wi-Mosquito-Net-Set/253639619614?hash=item3b0e19741e:m:mIHN9kGpsMWHPShA5HQi3uw&var=552873885342',
 'https://www.ebay.com/itm/Double-Sideds-Inflatable-Sleep-Pillow-Mat-Cushion-For-Camping-Picnic-Travel-Soft/132279480734?hash=item1ecc78b19e:m:m0v5IfPKuZEtGejjRdB1hww&var=431607141476',
 'https://www.ebay.com/itm/Ultralight-Compact-Folding-Aluminium-Alloy-Camping-Tent-Cot-Bed-with-Carry-Bag/123248048822?hash=item1cb227fab6:g:kEQAAOSwLYRbSFff']

Here we will define the job, the job (which also most likely is a function) itself might depend on some other funtions

In [9]:
def sale_rate_tables(table, table_no = 5):#function required for getting availability of items
    rows =  table[table_no].find_all('tr')
    sold_quantity = []
    date_of_sale = []
    for i in range(1, len(rows)-2):
        quantity = np.int(rows[i].find_all('td')[-3].text)
        sold_quantity.append(quantity)
        date_sale = parser.parse(rows[i].find_all('td')[-2].text)
        date_of_sale.append(date_sale)

    sale_duration = (date_of_sale[0]-date_of_sale[-1]).days
    total_quantity_sold = np.array(sold_quantity).sum()
    print(total_quantity_sold, sale_duration)
    sale_rate = total_quantity_sold/sale_duration
    return sale_rate
    

In [49]:
def job_get_item_details(url_list):
    sellerfeedback = []
    item_available = []
    price = []
    item_currency = []
    item_description = []
    keyword = []
    seller_name = []
    currency = []
    quantity_available = []
    for url in url_list:
        print(url)
        response = get(url)
        first_soup = BeautifulSoup(response.text, 'html.parser')# get the html soup
        heading = first_soup.find_all('h1', class_ = "it-ttl")#find the item description
        seller = first_soup.find_all('span', class_ = "mbg-nw")#find the seller name
        seller_feedback = first_soup.find_all('span', class_ = "mbg-l")#get the seller feedback
        item_description.append(heading[0].text[16:])
        seller_name.append(seller[0].text)
        sellerfeedback.append(np.int(seller_feedback[0].text[15:-2]))
        item = first_soup.find_all('span', itemprop = "name")
        available_quantity = first_soup.find_all('span', id = "qtySubTxt")
        retail_price = first_soup.find_all('span', id = "prcIsum")#find the retail price
        discount_price = first_soup.find_all('span', id = "mm-saleDscPrc")
        try:#append retail price, look for error
            y=np.float(retail_price[0].attrs['content'])
            price.append(y)
            print('The price is {}'.format(y))
        except:#else look for discounted price if any
            z  = discount_price[0].text
            m = np.float(''.join(i for i in z if (i.isdigit() or i=='.')))
            price.append(m) 
            print('The discounted price is {}'.format(m))
        try:#append currency 
            price_string = retail_price[0].text.replace('.','')
            currency_1 = ''.join([i for i in price_string if not i.isdigit()])
            currency.append(currency_1)
            print('The currency is {}'.format(currency_1))
        except:#append currency for discounted price
            price_string = discount_price[0].text.replace('.','')
            currency_1 = ''.join([i for i in price_string if not i.isdigit()])
            currency.append(currency_1)
            print('The currency is {}'.format(currency_1))
        try:
            x = available_quantity[0].text
            quantity = [int(s) for s in x.split() if s.isdigit()][0]
            quantity_available.append(quantity)
            if quantity>0:
                item_available.append(1)
            else:
                item_available.append(0)
            print('Available quantity is : {}'.format(quantity))
        except:
            quantity_available.append(np.nan)
            item_available.append(0)
        try:
            keyword.append(item[4].text)#get the keyword
            print(item[4].text)
        except:
            if len(item)>0:
                keyword.append(item[-2].text)
                print(item[-2].text)
            else:
                keyword.append('Not available')
                print('Key word not available')
        
    filename = 'item_availability.csv'
    dfitems = {'Keyword':keyword, 'seller name':seller_name,
               'Price':price, 'Currency':currency, str(date.today()):item_available}
    return pd.DataFrame(dfitems, columns=dfitems.keys())

In [50]:
item_availability = job_get_item_details(item_urls)

https://www.ebay.com/itm/Inflatable-Air-Sofa-Bed-Lazy-Sleeping-Camping-Bag-Beach-Hangout-Couch-Windbed/292566375653?epid=715012867&hash=item441e509ce5:m:mSeLLE_HXu8OCpTWY6uyAZA&var=591377547355
The discounted price is 8.99
The currency is US $
Available quantity is : 10
Sleeping Bags
https://www.ebay.com/itm/Double-Self-Inflating-Camping-Roll-Mat-Camp-Bed-Inflatable-Sleeping-Mattress/253747098737?hash=item3b14817471:g:brEAAOSwZrdbRxom
The price is 4.99
The currency is US $
Available quantity is : 10
Mattresses & Pads
https://www.ebay.com/itm/New-Double-Outdoor-Person-Travel-Camping-Hanging-Hammock-Bed-Wi-Mosquito-Net-Set/253639619614?hash=item3b0e19741e:m:mIHN9kGpsMWHPShA5HQi3uw&var=552873885342
The price is 4.99
The currency is US $
Available quantity is : 129
Sporting Goods
https://www.ebay.com/itm/Double-Sideds-Inflatable-Sleep-Pillow-Mat-Cushion-For-Camping-Picnic-Travel-Soft/132279480734?hash=item1ecc78b19e:m:m0v5IfPKuZEtGejjRdB1hww&var=431607141476
The price is 1.49
The currency 

In [51]:
item_availability

,Keyword,seller name,Price,Currency,2018-07-15
0,Sleeping Bags,xianghuietchco.ltd,8.99,US $,1
1,Mattresses & Pads,klcollins23,4.99,US $,1
2,Sporting Goods,2011always-for-you,4.99,US $,1
3,Mattresses & Pads,lins-bay,1.49,US $,1
4,Cots,rugweavers101,14.99,C $,1


In [40]:
str(datetime.now())

'2018-07-15 17:58:34.529089'

In [45]:
date.today()

datetime.date(2018, 7, 15)